In [1]:
Q = 1234567891011

In [2]:
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(1033706424581, 29831229452, 171030237003)

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(x)

In [7]:
decrypt(a, b, c)

25

In [8]:
x = encrypt(25)
y = encrypt(5)
print(x)
print(y)

(-412926624412, -909824740646, 88183474072)
(-386682644683, -762708769803, 1149391414491)


In [9]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [10]:
decrypt(*add(x,y))

30

SMPC Using PySyft

In [12]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/opt/conda/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [14]:
x = torch.tensor([25])
x

tensor([25])

In [15]:
encrypted_x = x.share(bob, alice, bill)

/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:79: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")
/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:91: UserWarning: Default args selected
  warnings.warn("Default args selected")


Encryption is as simple as taking any PySyft tensor and calling .share(). Decryption is as simple as calling .get() on the shared variable

In [16]:
encrypted_x.get()

tensor([25])

In [17]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [18]:
z = x + y
z.get()

tensor([30])

For multiplication we need an additional party who is responsible for consistently generating random numbers (and not colluding with any of the other parties).

In [19]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [24]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:79: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")
/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:91: UserWarning: Default args selected
  warnings.warn("Default args selected")


In [25]:
z = x * y
z.get()

tensor([125])